# How to run SPAMM

The only required inputs to run `SPAMM` are:
1. A source AGN spectrum
2. A list of which components to fit

However, there are numerous optional input arguments available. Since there are dozens of optional parameters, they have been separated out into a configuration file, `parameters.yaml`. An example default file is provided in `spamm/utils/parameters.yaml`. Users will want to modify this file or create their own with the same format. Any parameters file can be specified at runtime- if one is not found, the default will be used.

## Getting Started

`SPAMM` can accept the required input spectrum in three different forms: 
1. `spamm.Spectrum` instance
2. `specutils.Spectrum1D` instance
3. tuple containing the wavelength and flux and optionally, the error on the flux

All three methods will be shown below.

The list of acceptable components to fit are listed below (they are case insensitive):
- `PL` (Nuclear Continuum)
- `FE` (Iron Forect)
- `HOST` (Host Galaxy)
- `BC` (Balmer Continuum)
- `BPC` (Balmer Pseudo Continuum)
- `CALZETTI_EXT` (Calzetti extinction curves)
- `SMC_EXT` (Small Magellanic Cloud extinction curves)
- `LMC_EXT` (Large Magellanic Cloud extinction curves)
- `MW_EXT` (Milky Way extinction curves)
- `AGN_EXT` (AGN extinction curves)

### From a `spamm.Spectrum` instance

In [ ]:
import numpy as np
import astropy.units as u
from spamm import spamm

In [ ]:
from astropy.modeling.powerlaws import PowerLaw1D

# Define the unit-less wavelength and flux
wavelength = np.arange(1000, 2000)

# Just generate a simple sample power law for a spectrum
pl = PowerLaw1D(amplitude=1.5e-17, x_0=1500, alpha=-2.0)
flux = pl(wavelength)

# Make the error 5% of the flux array 
flux_error = flux * 0.05

In [ ]:
from spamm.Spectrum import Spectrum

# If no wavelength and flux units are provided upon instantiation, 
# they will be retrieved from the parameters.yaml file.
inspectrum = Spectrum(spectral_axis=wavelength, flux=flux, flux_error=flux_error,
                      spectral_axis_unit=u.AA, flux_unit=u.Unit('erg cm-2 s-1 AA-1'))

# Let's fit only the Nuclear Continuum component.
spamm_out = spamm.spamm(complist=["PL"], inspectrum=inspectrum)

### From a `specutils.Spectrum1D` instance
The most important thing to note when starting from a `specutils.Spectrum1D` object is that `Spectrum1D` requires the wavelength and flux arrays to be `astropy.unit.Quantity` objects. While an input uncertainty is an optional parameter when instantiating `Spectrum1D`, it is required when instantiating a `spamm.Spectrum` object and therefore should be passed into `Spectrum1D` before using `spamm`. 
`Spectrum1D` requires the uncertainty to be an `astropy.nddata.NDUncertainty` object.

In [ ]:
from specutils import Spectrum1D
from astropy.nddata import StdDevUncertainty

In [ ]:
# specutils.Spectrum1D objects must be instantiated with units.
wavelength_q = wavelength * u.AA
flux_q = flux * u.Unit('erg cm-2 s-1 AA-1')
flux_error_q = flux_q * 0.05
# Explicit array and unit arguments are shown below to illustrate how they are passed in.
uncertainty = StdDevUncertainty(flux_error_q.value, unit=flux_error_q.unit)

In [ ]:
inspectrum = Spectrum1D(spectral_axis=wavelength_q, flux=flux_q, uncertainty=uncertainty)
spamm_out = spamm.spamm(complist=["PL"], inspectrum=inspectrum)

### From a tuple

In [ ]:
# Create a tuple with the wavelength, flux, and flux error arrays. 
intup = (wavelength, flux, flux_error)

In [ ]:
# Note that when the input spectrum is defined as a tuple, 
# the wavelength and flux units will be retrieved from the parameters file.
spamm_out = spamm.spamm(complist=["PL"], inspectrum=intup)

### Specifying an input parameters file
An input `parameters.yaml` can be specified on runtime using the `par_file` argument. If one is not specified, the default file will be used.

In [ ]:
spamm_out = spamm.spamm(complist, inspectrum, par_file="/path/to/my/parameters.yaml")